# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

In [1]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Volume, \
                        Cell, Vector2, Vector3, Quaternion
import nglview

be = BioExplorer('localhost:5000')

Failed to display Jupyter Widget of type _ColormakerRegistry . 
 
 If you're reading this message in the Jupyter Notebook or JupyterLab Notebook, it may mean
 that the widgets JavaScript is still loading. If this message persists, it
 likely means that the widgets JavaScript library is either not installed or
 not enabled. See the Jupyter
 Widgets Documentation for setup instructions.
 
 
 If you're reading this message in another frontend (for example, a static
 rendering on GitHub or NBViewer ),
 it may mean that your frontend doesn't currently support widgets.

In [2]:
status = be.reset()

### Resources

In [3]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

### Configuration

In [4]:
# Proteins
debug = False

protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
if debug:
    protein_representation = be.REPRESENTATION_DEBUG
protein_load_hydrogen = False

## Camera

In [5]:
status = be.core_api().set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[1.09, -3.343, 119.226],
    target=[1.09, -3.343, 12.059])

## Scene description

#### Surfactant-D

In [6]:
nglview.show_file(surfactant_head_source)

ValueError: you must provide file extension if using file-like object or text content

In [ ]:
nglview.show_file(surfactant_branch_source)

In [ ]:
def add_surfactant_d(name, position, rotation, random_seed):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, rotation=rotation,
        random_seed=random_seed)

add_surfactant_d(
    name='Surfactant-D 1', random_seed=1,
    position=Vector3(), rotation=Quaternion(0, 1, 0, 0))

## Materials

In [ ]:
be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)

for model in be.core_api().scene.models:
    model_id = model['id']
    model_name = model['name']
    if be.NAME_COLLAGEN in model_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)
        print(model_name)
        palette = list()
        emissions = list()
        for i in range(nb_materials):
            palette.append([1,1,1])
            emissions.append(0.2)
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette,
            emissions=emissions
        )
be.core_api().scene.commit()

## Rendering settings

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 1.0
params.soft_shadows = 0.3
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Snapshot

In [ ]:
from braynsmediamaker import MovieMaker
mm = MovieMaker(be.core_api())

In [ ]:
mm.create_snapshot(
    path='/tmp/surfactant-a.png',
    samples_per_pixel=64, size=[512, 512])